# Instructions
1. Click on `Runtime > Run all` from the top menu.
1. Copy the output at the bottom of the page; you can select it all, or click it and then press `CTRL + A`.
1. Go to the [state spreadsheet](https://docs.google.com/spreadsheets/d/1FtEIDMiKOgcfeot9M6khWzITd-7seWQ4EdMkzG6fmw8/edit#gid=1094833013), right-click the first empty cell at the bottom and select `Paste special > Paste values only`.

Please make sure no previous data is overwritten!

Also verify that the date is correct, since the current date is automatically filled in but the last updated time could be from several days prior.

In [ ]:
import datetime
import re
import pandas as pd

SCHOOL_DATA = 'https://coronavirus.ohio.gov/static/dashboards/school_reporting.csv'
DISTRICT_METADATA = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSfTEYXDiLrEw42lUDnhu7peCxqjx_9I-8zMBmpa1QdlmWCJGA1R8W21vNZUrAuu8HoTynXk7eo5Obg/pub?gid=243950450&single=true&output=csv'

# Load case data and metadata for this state
df = pd.read_csv(SCHOOL_DATA, dtype=str)
metadata = pd.read_csv(DISTRICT_METADATA, dtype=str)
metadata = metadata[metadata['state'] == 'OH'] 

# Match the data with our known district identifiers
df['district_name'] = df['school_or_school_district']
metadata['county'] = metadata['county_name'].str.replace(' County', '')
df = df.merge(metadata, how='inner')

# Add a date and website fields to the data
df['date'] = datetime.datetime.now().date().isoformat()
df['website'] = SCHOOL_DATA

# Output columns in the same order as our data
df = df[[
  "date",
  "district_id",
  "district_name",
  "student_cases_new",
  "student_cases_cumulative",
  "staff_cases_new",
  "staff_cases_cumulative",
  "website",
]]

# Replace unknown data with nulls
for column in [col for col in df.columns if '_new' in col or '_cumulative' in col]:
  df[column] = df[column].apply(lambda x: x if re.match(r'\d+', x) else None)

print(df.to_csv(sep='\t', index=False, header=False))